In [1]:
import rootpath
root_dir = '/home/yiyi/Documents/masterarbeit/'
import ast

In [2]:
import os
import pandas as pd
import json
from more_itertools import flatten

In [3]:
import rootpath

In [10]:
category_dict =[('renovation_related-entertainment', 1),
 ('renovation_related-beach', 1),
 ('renovation_related-venue', 1),
 ('renovation_related-atmosphere', 2),
 ('renovation_related-restaurant', 3),
 ('renovation_related-reception', 3),
 ('renovation_related-pool', 4),
 ('renovation_related-facility', 10),
 ('entertainment', 13),
 ('venue', 23),
 ('beach', 24),
 ('family', 30),
 ('parking', 42),
 ('fitness', 47),
 ('price', 55),
 ('atmosphere', 55),
 ('pool', 86),
 ('renovation_related-room', 94),
 ('facility', 108),
 ('restaurant', 119),
 ('reception', 126),
 ('room', 179)]

In [4]:
required = ["room", "reception", "restaurant", "facility", "renovation_related-room", "pool"]

In [5]:
root_dir = rootpath.detect()

In [6]:
root_dir

'/home/yiyi/Documents/masterarbeit/CPD'

In [7]:
data_dir = "data/spellchecked"
json_files = "data/prepared_data_20200615/json_files"
output_dir = "data/jsonfiles_db"

In [8]:
from collections import defaultdict

In [9]:
# process one file 
def get_uid_sentence(file):
    uids = list()
    uid_sentence = dict()
    with open(file) as reader:
        data = json.load(reader)
    count =0
    for idx, l in data["sentences"].items():
        # idx : rupture point.
        for elem in l :
            # date, uid, lemma, sentence, sentiment, cpt.
            d = ast.literal_eval(elem[2])
            uid_sentence[count] = (elem[0], elem[1], d, elem[3], elem[4], idx)
            uids.append(elem[1])
            count+=1
    dates = data["dates_periods"]
    
    return uid_sentence, uids, dates
        

In [10]:
filename = "atmosphere_14#8d0d0202-48c8-4d09-bdb0-b3a75bec23a9.json"

In [11]:
uid_sentence, uids, dates = get_uid_sentence(os.path.join(root_dir, json_files, filename))

In [12]:
uid_sentence[0]

('2015-07-09',
 '13644ce0-7b48-4c12-ad36-f1f87b0b84d9',
 ['professional', 'aspect', 'work', 'true', 'advocate', 'atlantis', 'brand'],
 'He is always very professional in all aspects of his work and a true advocate for the Atlantis brand',
 0.498506247997284,
 '0')

In [36]:

def get_new_json_content(filename, file_id):
    review_list = []
    
    filepath = os.path.join(root_dir, json_files, filename)
    print(filepath)
    # get from json file.
    if os.path.isfile(filepath) and not filename.startswith("."):
        sents, uids, dates  = get_uid_sentence(filepath)
        filename0 = filename.split('_')[0]
        print(filename0) 
        name = filename.split('#')[0] # "renovation_related-room_72"
        name0 = name.rsplit('_',1)[0]
        print("name ==>", name)
        print("name0 ===>", name0)
        if name0 in required:
            if filename0 in topics:
                categories =[filename0]

            if 'renovation_related-' in name:
                categories = [name0.split('-')[-1], "renovation"]
            print(categories)
            keywords = list(flatten([topics[x] for x in categories]))

            # get the filename to find the dataframe.
            filename_ = filename.split('_')[-1].replace('.json', '')

            csv_filepath = os.path.join(root_dir, data_dir, filename_)
            print(csv_filepath)

            # get the csv.
            df = pd.read_csv(csv_filepath)
            rows = df[df['uid'].isin(uids)]

            # regroup the sentences into reviews.
            groups = rows.groupby('uid')['sentence'].apply(list).reset_index(name ='new')

            uid_sents_list =dict()
            for uid, new in zip(groups.uid, groups.new ):
                uid_sents_list[uid] = new

            for idx, sentence in sents.items():
                # uid, sent, date, cpt, category = sentence
                date, uid, lemma, sent, sentiment, cpt = sentence
                sent_list = uid_sents_list[uid]
                try:
                    review = '. '.join(sent_list)+'.'

                    sent_lower= sent.lower()
                    hashtags = list(set(lemma).intersection(set(keywords)))

                    offset = (review.find(sent), review.find(sent)+len(sent)+1)

                    # print(sent[offsets[0][0]:offsets[0][1]])

                    review_list.append({
                        "id":idx,
                        "uid":uid,
                        "review":review, 
                        "offset_sent":offset,
                        "hashtags":hashtags,
                        "date":date,
                        "cpt": cpt, 
                    })
    #                 print("sentence:", sent)
    #                 print("review:", review)
    #                 print("offset:", offset, " len:", len(review))
    #                 print("offset topics:", offsets)
    #                 print('sucess')
    #                 print('*'*40)
                except Exception:
                    print(Exception)

            return {"id": file_id, "name":name, "categories":categories, "dates_periods":dates, "review_list": review_list}



In [37]:
filename="renovation_related-room_72#200a52ac-6026-403f-81c6-97c19df5188c.json"

In [38]:
reviews = get_new_json_content(filename,1)


/home/yiyi/Documents/masterarbeit/CPD/data/prepared_data_20200615/json_files/renovation_related-room_72#200a52ac-6026-403f-81c6-97c19df5188c.json
renovation
name ==> renovation_related-room_72
name0 ===> renovation_related-room
['room', 'renovation']
/home/yiyi/Documents/masterarbeit/CPD/data/spellchecked/72#200a52ac-6026-403f-81c6-97c19df5188c


In [39]:
required

['room',
 'reception',
 'restaurant',
 'facility',
 'renovation_related-room',
 'pool']

In [40]:
exceptions =[]
reviews = []
count=0
for file in os.listdir(os.path.join(root_dir, json_files)):

    outputfile = os.path.join(root_dir, output_dir, file)
    json_dict = get_new_json_content(file, count)

    if json_dict is not None and len(json_dict)>=1:
        reviews.append(json_dict)
        count+=1

/home/yiyi/Documents/masterarbeit/CPD/data/prepared_data_20200615/json_files/facility_136#4ea7ff31-71d7-4532-b8ca-3ab08664c3d5.json
facility
name ==> facility_136
name0 ===> facility
['facility']
/home/yiyi/Documents/masterarbeit/CPD/data/spellchecked/136#4ea7ff31-71d7-4532-b8ca-3ab08664c3d5
<class 'Exception'>
/home/yiyi/Documents/masterarbeit/CPD/data/prepared_data_20200615/json_files/reception_281#01aabe1f-19d6-48c6-a186-4211537a66b8.json
reception
name ==> reception_281
name0 ===> reception
['reception']
/home/yiyi/Documents/masterarbeit/CPD/data/spellchecked/281#01aabe1f-19d6-48c6-a186-4211537a66b8
/home/yiyi/Documents/masterarbeit/CPD/data/prepared_data_20200615/json_files/restaurant_434#d167ce8b-a01c-4cec-ae99-5f57a40125a0.json
restaurant
name ==> restaurant_434
name0 ===> restaurant
['restaurant']
/home/yiyi/Documents/masterarbeit/CPD/data/spellchecked/434#d167ce8b-a01c-4cec-ae99-5f57a40125a0
<class 'Exception'>
/home/yiyi/Documents/masterarbeit/CPD/data/prepared_data_20200615/

In [28]:
len(exceptions)

0

In [41]:
with open(root_dir+'/'+output_dir+'/0615_db.json', 'w') as file:
    json.dump({ "annotations":[], "reviews":reviews}, file)

In [65]:
with open(root_dir+'/'+output_dir+'/0615_db.json') as file:
    data = json.load(file)

In [72]:
required

['room',
 'reception',
 'restaurant',
 'facility',
 'renovation_related-room',
 'pool']

In [42]:
change_points = defaultdict(int)

for item in reviews:
    name = item["name"].rsplit("_", 1)[0]
    dates_periods = item["dates_periods"]
    change_points[name]+=len(dates_periods)-1

In [43]:
change_points

defaultdict(int,
            {'facility': 141,
             'reception': 193,
             'restaurant': 157,
             'pool': 138,
             'room': 253,
             'renovation_related-room': 114})

In [44]:
sum(change_points.values())

996

In [31]:
reviews[0]

{'id': 0,
 'name': 'facility_136',
 'categories': ['facility'],
 'dates_periods': [['2015-01-01', '2016-12-04'], ['2016-12-07', '2020-04-14']],
 'review_list': [{'id': 0,
   'uid': '52fac95c-1555-4dc5-b639-184f71ccc191',
   'review': 'Great location breakfast. Loved the staff convenient location lobby decor and breakfast buffet selection. My only complaint is that I wish our room matched the updated lobby common areas. I found the furniture to be well worn and the central A C dated and hard to get and maintain a desired temperature.',
   'offset_sent': (106, 187),
   'hashtags': ['lobby'],
   'date': '2015-01-24',
   'cpt': '0'},
  {'id': 1,
   'uid': '0fa7c2a5-5f3e-4f41-8ab9-f2f0a6865e03',
   'review': 'Bristow DMB concert stayp. Very convenient location in the DC Metro area. My wife and I needed a nice hotel close to Bristow for a Concert we were attending with friends. Our friends eventually decieded to stay at the same hotel. Only gripe I had was that the Hyat hotels do not seem to

In [11]:
reviews = data["reviews"]

In [37]:
for review in reviews:
    if review is None: 
        print(review)

In [16]:
topics = {
  "room": [
    "analog",
    "balcony",
    "basin",
    "bath",
    "bathroom",
    "bathtub",
    "bedroom",
    "bidet",
    "bottled",
    "carpeting",
    "casita",
    "closet",
    "commode",
    "courtyard",
    "cubicle",
    "cupboard",
    "deck",
    "delux",
    "deluxe",
    "ensuite",
    "facing",
    "faucet",
    "fitting",
    "flooring",
    "foxtel",
    "fridge",
    "furnishing",
    "furniture",
    "guestroom",
    "hbo",
    "hdtv",
    "lampshade",
    "lanai",
    "mini",
    "minibar",
    "mismatched",
    "oceanfront",
    "oceanview",
    "outlook",
    "overlook",
    "overlooking",
    "paintwork",
    "panorama",
    "panoramic",
    "patio",
    "porch",
    "porthole",
    "projector",
    "rainshower",
    "refrigerator",
    "restock",
    "restocking",
    "room",
    "rooms",
    "seaview",
    "sensor",
    "shower",
    "showerhead",
    "showering",
    "sink",
    "skyline",
    "soda",
    "sprayer",
    "stock",
    "studio",
    "suite",
    "suites",
    "swivel",
    "tattered",
    "television",
    "telly",
    "terrace",
    "tub",
    "tv",
    "tvs",
    "unit",
    "upholstery",
    "vanity",
    "veranda",
    "view",
    "views",
    "villa",
    "washbasin",
    "washroom",
    "wc",
    "wifi",
    "window"
  ],
  "restaurant": [
    "abalone",
    "appetizer",
    "bagel",
    "bagels",
    "bakery",
    "bar",
    "bars",
    "barthe",
    "bistro",
    "breakfast",
    "breakfasts",
    "breakfastthe",
    "brekkie",
    "brekky",
    "brownie",
    "buffet",
    "burrito",
    "cafe",
    "caf\u00e9",
    "canape",
    "canap\u00e9",
    "candy",
    "cheesecake",
    "congee",
    "cupcake",
    "deli",
    "dessert",
    "dine",
    "diner",
    "dining",
    "dinner",
    "dish",
    "donut",
    "dunkin",
    "eatery",
    "eating",
    "einstein",
    "entree",
    "entr\u00e9e",
    "flay",
    "fritti",
    "gelato",
    "grille",
    "hor",
    "laksa",
    "lumiere",
    "lunch",
    "lunchtime",
    "macaroon",
    "marketplace",
    "meal",
    "menu",
    "mousse",
    "muffin",
    "munchie",
    "nibble",
    "ogrille",
    "osheas",
    "paella",
    "pasta",
    "pastry",
    "plate",
    "platter",
    "pub",
    "refreshment",
    "restaraunt",
    "restaurant",
    "restaurants",
    "resteraunt",
    "restraint",
    "resturaunt",
    "ricos",
    "risotto",
    "roombreakfast",
    "salad",
    "sauce",
    "selection",
    "shisha",
    "shuckers",
    "skybar",
    "snack",
    "snacks",
    "soda",
    "soup",
    "speciality",
    "specialty",
    "starter",
    "steakhouse",
    "substitution",
    "sundara",
    "supper",
    "takeaway",
    "takeout",
    "tart",
    "tepanyaki",
    "tiramisu",
    "travis",
    "truffle",
    "veg",
    "vegetable",
    "vegetarian"
  ],
  "facility": [
    "accessory",
    "accommodation",
    "amenities",
    "amenity",
    "amentie",
    "amentitie",
    "ammenitie",
    "ammentie",
    "atrium",
    "ballroom",
    "button",
    "chandelier",
    "downstairs",
    "elavator",
    "elevator",
    "elevators",
    "entrance",
    "entryway",
    "equipment",
    "escalator",
    "facilities",
    "facilitiesthe",
    "facility",
    "faciltie",
    "foyer",
    "gym",
    "hall",
    "hallway",
    "infrastructure",
    "keycard",
    "lift",
    "lobby",
    "necessity",
    "reception",
    "stair",
    "staircase",
    "stairway",
    "stairwell",
    "toiletry"
  ],
  "pool": [
    "float",
    "grotto",
    "hottub",
    "jacuzzi",
    "lap",
    "lounging",
    "poll",
    "pool",
    "pools",
    "poolside",
    "poolthe",
    "shallow",
    "sunbathing",
    "sunning",
    "swim",
    "swimmer",
    "swimming",
    "swimmingpool",
    "wade"
  ],
  "family": [
    "cation",
    "decompress",
    "fam",
    "family",
    "getaway",
    "grandchild",
    "grandkid",
    "grandparent",
    "group",
    "holiday",
    "nephew",
    "outing",
    "parent",
    "spouse",
    "staycation",
    "teen",
    "teenager",
    "trip",
    "vaca",
    "vacation",
    "vacay"
  ],
  "price": [
    "charge",
    "cost",
    "costly",
    "dollar",
    "euro",
    "expense",
    "expensive",
    "markup",
    "overprice",
    "pay",
    "price",
    "pricey",
    "pricing",
    "rate",
    "surcharge",
    "tariff",
    "tax",
    "taxis",
    "upcharge"
  ],
  "atmosphere": [
    "aesthetic",
    "ambiance",
    "ambience",
    "architecture",
    "aspect",
    "atmosphere",
    "buzzy",
    "chic",
    "classical",
    "conclusion",
    "ending",
    "enviroment",
    "environment",
    "experience",
    "expierence",
    "hip",
    "memory",
    "mondrian",
    "refined",
    "scenario",
    "scene",
    "setting",
    "sophisticated",
    "staying",
    "surrounding",
    "swanky",
    "trendy",
    "unpretentious",
    "vibe",
    "vibrant"
  ],
  "fitness": [
    "activities",
    "activity",
    "aerobic",
    "aerobics",
    "buggy",
    "cardio",
    "copperhead",
    "elliptical",
    "exercise",
    "fitness",
    "golf",
    "golfer",
    "golfing",
    "gym",
    "gymn",
    "gymnasium",
    "hammam",
    "hydro",
    "hydrotherapy",
    "mani",
    "obstacle",
    "pedi",
    "polo",
    "putt",
    "rec",
    "recreation",
    "riding",
    "rockerfeller",
    "sauna",
    "spa",
    "spar",
    "sport",
    "sports",
    "steamroom",
    "technogym",
    "tennis",
    "therapy",
    "tpc",
    "trampoline",
    "treadmill",
    "tricycle",
    "volleyball",
    "watersport",
    "wellness",
    "workout"
  ],
  "transport": [
    "bus",
    "chauffeur",
    "closeness",
    "doorstep",
    "downtown",
    "fishermans",
    "jitney",
    "limo",
    "limousine",
    "locale",
    "locality",
    "locate",
    "location",
    "minibus",
    "pickup",
    "position",
    "proximity",
    "shuttle",
    "tram",
    "transit",
    "transporation",
    "transport",
    "transportaion",
    "transportation",
    "trolley",
    "trolly",
    "valletta",
    "van"
  ],
  "beach": [
    "bay",
    "beach",
    "beaches",
    "beachfront",
    "beech",
    "coastline",
    "cove",
    "gulf",
    "harbor",
    "harbour",
    "intercoastal",
    "lagoon",
    "lake",
    "mountain",
    "ocean",
    "oceanfront",
    "oceanview",
    "pools",
    "reef",
    "sand",
    "sandy",
    "scenic",
    "sea",
    "shore",
    "sunbathing",
    "turquoise",
    "vineyard",
    "waterway"
  ],
  "entertainment": [
    "arcade",
    "bah",
    "bahamar",
    "casino",
    "casinos",
    "dancing",
    "disco",
    "dj",
    "entertaining",
    "entertainment",
    "karaoke",
    "klcc",
    "mall",
    "nightlife",
    "palladium",
    "performance",
    "performer",
    "singer",
    "souk",
    "southbank"
  ],
  "venue": [
    "amphitheater",
    "arena",
    "atrium",
    "auditorium",
    "ballroom",
    "banquet",
    "breakout",
    "campus",
    "conference",
    "convention",
    "event",
    "events",
    "expo",
    "foyer",
    "function",
    "fundraiser",
    "fundraising",
    "gala",
    "gathering",
    "locale",
    "luncheon",
    "meeting",
    "meetings",
    "networking",
    "organizer",
    "seminar",
    "sporting",
    "staircase",
    "tent",
    "venue",
    "workshop"
  ],
  "parking": [
    "carpark",
    "carparking",
    "dealership",
    "garage",
    "locationparking",
    "ncp",
    "park",
    "parkand",
    "parke",
    "parked",
    "parking",
    "parks",
    "rental",
    "vehicle",
    "waterpark"
  ],
  "reception": [
    "attendant",
    "attendent",
    "bellboy",
    "bellhop",
    "bellman",
    "checking",
    "conceirge",
    "concerige",
    "conciege",
    "concierge",
    "concierges",
    "consierge",
    "crew",
    "delivery",
    "desk",
    "employee",
    "escort",
    "frontdesk",
    "inservice",
    "insider",
    "navigator",
    "personell",
    "personnel",
    "porter",
    "reception",
    "receptionist",
    "registration",
    "reservationist",
    "responsiveness",
    "roomservice",
    "service",
    "services",
    "servicethe",
    "servicewe",
    "servicing",
    "staff",
    "staffer",
    "staffthe",
    "team",
    "thoroughness",
    "waitstaff"
  ],
  "renovation": [
    "caulking",
    "dismissal",
    "facelift",
    "hmc",
    "maintenance",
    "makeover",
    "manej",
    "modernisation",
    "modernise",
    "modernization",
    "modernize",
    "overhaul",
    "recondition",
    "reconstruction",
    "redecorate",
    "redecoration",
    "redesign",
    "redo",
    "redone",
    "refit",
    "reform",
    "refresh",
    "refurb",
    "refurbish",
    "refurbished",
    "refurbishing",
    "refurbishment",
    "remodel",
    "remodeling",
    "renew",
    "renewal",
    "rennovation",
    "reno",
    "renovate",
    "renovated",
    "renovation",
    "repaint",
    "repair",
    "restoration",
    "resurfacing",
    "revamp",
    "revision",
    "scrubbing",
    "spruce",
    "tuttle",
    "update",
    "updating",
    "upgrading",
    "upkeep"
  ]
}